In [1]:
import tensorflow as tf

# Variable用法

In [2]:
var1 = tf.Variable(1.0, name = 'firstvar')
print("var1: ", var1.name)

Instructions for updating:
Colocations handled automatically by placer.
var1:  firstvar:0


In [3]:
var1 = tf.Variable(2.0 , name='firstvar')
print ("var1:",var1.name)

var1: firstvar_1:0


在Variable定义时没有指定名字，则系统会自动生成名字Variable：0

In [4]:
var2 = tf.Variable(3.0 )
print ("var2:",var2.name)

var2: Variable:0


In [5]:
var2 = tf.Variable(4.0 )
print ("var2:",var2.name)

var2: Variable_1:0


当定义var1两次时，内存会生成两个var1,但是只有最后定义的var1对Session有效（var1 = 2.0）;var2同理。

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("var1=",var1.eval())
    print("var2=",var2.eval())

var1= 2.0
var2= 4.0


# get_variable用法

In [7]:
get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.3))
print ("get_var1:",get_var1.name)

get_var1: firstvar_2:0


定义第二个get_variable时崩溃，因为get_variable只能定义一次指定名称的变量。

In [8]:
get_var1 = tf.get_variable("firstvar",[1], initializer=tf.constant_initializer(0.4))
print ("get_var1:",get_var1.name)

ValueError: Variable firstvar already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


In [9]:
get_var1 = tf.get_variable("firstvar1",[1], initializer=tf.constant_initializer(0.4))
print ("get_var1:",get_var1.name)

get_var1: firstvar1:0


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("get_var1=",get_var1.eval())

get_var1= [0.4]


# variable_scope指定变量作用域

使用variable_scope隔离变量作用域，此时可以在不同作用域内创建名字相同的变量。

In [11]:
with tf.variable_scope("test1", ):
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

In [12]:
with tf.variable_scope("test2"):
    var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

In [13]:
print ("var1:",var1.name)
print ("var2:",var2.name)

var1: test1/firstvar:0
var2: test2/firstvar:0


嵌套不同的作用域：

In [16]:
with tf.variable_scope("test3", ):
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    
    with tf.variable_scope("test4"):
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

In [17]:
print ("var1:",var1.name)
print ("var2:",var2.name)

var1: test3/firstvar:0
var2: test3/test4/firstvar:0


# reuse参数实现共享变量

In [18]:
with tf.variable_scope("test5", ):
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    
    with tf.variable_scope("test6"):
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
        
print ("var1:",var1.name)
print ("var2:",var2.name)

var1: test5/firstvar:0
var2: test5/test6/firstvar:0


将variable_scope的reuse设置为True，则可以实现对变量的共享。var1和var3共享变量，var2和var4共享变量。

In [19]:
with tf.variable_scope("test5",reuse=True ):
    var3= tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    with tf.variable_scope("test6"):
        var4 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

print ("var3:",var3.name)
print ("var4:",var4.name)

var3: test5/firstvar:0
var4: test5/test6/firstvar:0


# 初始化共享变量作用域

In [20]:
with tf.variable_scope("test7", initializer=tf.constant_initializer(0.4) ): #将test7作用域初始化为0.4
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32) #var1没有初始化，但是继承test7的值
    
    with tf.variable_scope("test8"): #test8嵌套在test7中，且未初始化
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32) #var2继承test7的值
        var3 = tf.get_variable("var3",shape=[2],initializer=tf.constant_initializer(0.3)) #var3初始化为0.3

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("var1=",var1.eval())
    print("var2=",var2.eval())
    print("var3=",var3.eval())

var1= [0.4 0.4]
var2= [0.4 0.4]
var3= [0.3 0.3]


# variable_scope的as用法

In [22]:
with tf.variable_scope("scope1") as sp:
     var1 = tf.get_variable("v", [1])

print("sp:",sp.name)    
print("var1:",var1.name)      

sp: scope1
var1: scope1/v:0


In [23]:
with tf.variable_scope("scope2"):
    var2 = tf.get_variable("v", [1])
    
    with tf.variable_scope(sp) as sp1:
        var3 = tf.get_variable("v3", [1])
          
        with tf.variable_scope("") : #tf.name_scope使用空字符则创建空白作用域
            var4 = tf.get_variable("v4", [1])
            
print("sp1:",sp1.name)  
print("var2:",var2.name)
print("var3:",var3.name)
print("var4:",var4.name)

sp1: scope1
var2: scope2/v:0
var3: scope1/v3:0
var4: scope1//v4:0


In [24]:
with tf.variable_scope("scope"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
        with tf.name_scope(""): #tf.name_scope使用空字符将作用域返回到顶层
            y = 1.0 + v
print("v:",v.name)  
print("x.op:",x.op.name)
print("y.op:",y.op.name)

v: scope/v:0
x.op: scope/bar/add
y.op: add
